In [3]:
import pandas as pd 
import numpy as np
from collections import  OrderedDict

# LS + Btree

In [45]:
def gen_path(solver, solver_hyper, regularizer, depth_lst, 
            remove_ratio_lst = [0.05, 0.1, 0.2], 
            purpose='leastsquare_btree/details', 
            loss='ls', task='btree', lammax=1.0, lam_shrink=1e-2, 
            ):
    all_path = []
    for depth in depth_lst:
        dataset = f'ls_dep:{depth}_num:2x_sparse:0.1'
        for remove_ratio in remove_ratio_lst:
            path = f'../{purpose}/{solver}_{loss}_{regularizer}/{dataset}_lam_shrink:{lam_shrink}_{task}_{lammax}_{depth}_True_{remove_ratio}_{-1.0}'
            for k,v in solver_hyper.items():
                if k == 'ipg_strategy':
                    parh += f'_{v}'
                else:
                    path += f'_{k}:{v}'
            path += '_stats.npy'
            all_path.append(path)
    return all_path

def create_hypers(solver, modify_dict={}):
    task_hypers_template = {
            'ProxGD':   OrderedDict({'proxgd_stepsize': 'linesearch'}),
            'ProxSAGA': OrderedDict({ 'ipg_strategy': 'linear_decay', 'seed':2003, 'proxsaga_lipcoef': 1.0,}),
            'ProxSVRG': OrderedDict({ 'ipg_strategy': 'linear_decay', 'seed':2003, 'proxsvrg_inner_repeat': 1, 'proxsvrg_lipcoef': 1.0}),
            'PStorm':   OrderedDict({ 'ipg_strategy': 'diminishing', 'seed':2003,'pstorm_stepsize': 'diminishing', 'pstorm_betak': -1.0, 'pstorm_lipcoef': 1.0}),
            'RDA':      OrderedDict({ 'ipg_strategy': 'diminishing', 'seed':2003, 'rda_stepconst': 1.0}),
            'SPStorm':  OrderedDict({ 'ipg_strategy': 'diminishing', 'seed':2003, 'spstorm_betak': -1.0, 'spstorm_zeta': 'dynanmic', 'spstorm_lipcoef': 1.0})
        }
    hypers = task_hypers_template[solver]
    for k,v in modify_dict.items():
        hypers[k] = v
    return hypers
def load_from_npy(all_path, selected_keys=['xend', 'nnz', 'nz', 'Fend', 'iteration', 'status']):
    ground_truth = {}
    for path in all_path:
        info = np.load(path, allow_pickle=True).item()
        depth, remove_ratio = path.split('True')[0].split("_")[-2], path.split('True')[1].split("_")[1]
        instance_name = f'{depth}_{remove_ratio}'
        ground_truth[instance_name] = {k:info[k] for k in selected_keys}
        if info['status'] != 0:
            print(f'Warning: {instance_name} is not solved, status: {info["status"]}')
    return ground_truth
        

In [46]:
# get the ground truth
solver = 'ProxGD'
solver_hyper = create_hypers(solver)
all_path = gen_path(solver, solver_hyper, regularizer='TreeOG', depth_lst=[11,12,13])
ground_truth = load_from_npy(all_path)